# Wine Reviews Data Cleaning

Refer to [Data Understanding](data_understanding.ipynb) for explanation of the data cleaning strategy.

**Libraries**

In [23]:
import os

# data manipulation libraries
import pandas as pd
import numpy as np

# custom project libraries
from utils.wine_review_utils import load_original_data, TITLE_PATTERN

# plotting libraries
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
sns.set_theme()

## Load the data from Kaggle

In [24]:
reviews = load_original_data()
reviews.info()
reviews.head()

<class 'pandas.core.frame.DataFrame'>
Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,None,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,None,None,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## Data Cleaning

The data cleaning strategy will be to:
1. select reviews from the top $50$ wine varieties,
1. select reviews from countries with more than $200$ reviews,
1. drop reviews missing `variety`
1. from a vintage newer than $2003$
1. drop duplicate `descriptions`
1. drop the following fields: `price`, `province`, `region_1`, `region_2`, `taster_twitter_handle`, and `designation`
1. set missing `taster_name` to **Unknown**

### Extract vintage year from Title and Keep wines from 2004 on

In [25]:
# extract the vintage year from the title field
cleaned_reviews = reviews.copy(deep=True)
cleaned_reviews['year'] = cleaned_reviews['title'].apply(lambda x: TITLE_PATTERN.findall(x)[-1] if TITLE_PATTERN.findall(x) else None)

# drop the rows with missing vintage year
cleaned_reviews.dropna(subset=['year'], inplace=True)
cleaned_reviews['year'] = cleaned_reviews['year'].astype(int)
# only keep the rows with vintage year newer than 2003
cleaned_reviews = cleaned_reviews.query('year > 2003')

### Drop Duplicate Reviews

In [26]:
cleaned_reviews.drop_duplicates(subset=['description'], inplace=True, keep='last')

### Keep reviews from countries and varieties with most wines reviewed

In [27]:
# determine the countries with more than 200 reviews
min_reviews = 200
top_countries = cleaned_reviews.country.value_counts().reset_index().query('count > @min_reviews').set_index('country').index
# determine the top 50 varieties with the most reviews
max_varieties = 50
varieties_with_many_reviews = cleaned_reviews.variety.value_counts().head(50)

# filter the reviews to include only the top 50 varietals and top 15 countries
mask = cleaned_reviews.country.isin(top_countries)
mask &= cleaned_reviews.variety.isin(varieties_with_many_reviews.head(max_varieties).index)
cols = ['winery', 'variety', 'year', 'country', 'price', 'taster_name', 'description', 'points']
cleaned_reviews[mask][cols].info()
cleaned_reviews[mask][cols]

<class 'pandas.core.frame.DataFrame'>
Index: 98460 entries, 0 to 129970
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   winery       98460 non-null  object 
 1   variety      98460 non-null  object 
 2   year         98460 non-null  int64  
 3   country      98460 non-null  object 
 4   price        91633 non-null  float64
 5   taster_name  78843 non-null  object 
 6   description  98460 non-null  object 
 7   points       98460 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 6.8+ MB


,winery,variety,year,country,price,taster_name,description,points
0,Nicosia,White Blend,2013,Italy,NaN,Kerin O’Keefe,"Aromas include tropical fruit, broom, brimston...",87
1,Quinta dos Avidagos,Portuguese Red,2011,Portugal,15.0,Roger Voss,"This is ripe and fruity, a wine that is smooth...",87
2,Rainstorm,Pinot Gris,2013,US,14.0,Paul Gregutt,"Tart and snappy, the flavors of lime flesh and...",87
3,St. Julian,Riesling,2013,US,13.0,Alexander Peartree,"Pineapple rind, lemon pith and orange blossom ...",87
4,Sweet Cheeks,Pinot Noir,2012,US,65.0,Paul Gregutt,"Much like the regular bottling from 2012, this...",87
...,...,...,...,...,...,...,...,...
129966,Dr. H. Thanisch (Erben Müller-Burggraef),Riesling,2013,Germany,28.0,Anna Lee C. Iijima,Notes of honeysuckle and cantaloupe sweeten th...,90
129967,Citation,Pinot Noir,2004,US,75.0,Paul Gregutt,Citation is given as much as a decade of bottl...,90
129968,Domaine Gresser,Gewürztraminer,2013,France,30.0,Roger Voss,Well-drained gravel soil gives this wine its c...,90
129969,Domaine Marcel Deiss,Pinot Gris,2012,France,32.0,Roger Voss,"A dry style of Pinot Gris, this is crisp with ...",90


### Set missing `taster_name` to **Unkown**

In [28]:
cleaned_reviews.loc[cleaned_reviews.taster_name.isna(), 'taster_name'] = 'Unknown'
cleaned_reviews[mask][cols].info()
cleaned_reviews[mask][cols].head()

<class 'pandas.core.frame.DataFrame'>
Index: 98460 entries, 0 to 129970
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   winery       98460 non-null  object 
 1   variety      98460 non-null  object 
 2   year         98460 non-null  int64  
 3   country      98460 non-null  object 
 4   price        91633 non-null  float64
 5   taster_name  98460 non-null  object 
 6   description  98460 non-null  object 
 7   points       98460 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 6.8+ MB


,winery,variety,year,country,price,taster_name,description,points
0,Nicosia,White Blend,2013,Italy,NaN,Kerin O’Keefe,"Aromas include tropical fruit, broom, brimston...",87
1,Quinta dos Avidagos,Portuguese Red,2011,Portugal,15.0,Roger Voss,"This is ripe and fruity, a wine that is smooth...",87
2,Rainstorm,Pinot Gris,2013,US,14.0,Paul Gregutt,"Tart and snappy, the flavors of lime flesh and...",87
3,St. Julian,Riesling,2013,US,13.0,Alexander Peartree,"Pineapple rind, lemon pith and orange blossom ...",87
4,Sweet Cheeks,Pinot Noir,2012,US,65.0,Paul Gregutt,"Much like the regular bottling from 2012, this...",87


## Repeat data cleaning using custom project library

In [29]:
from utils.wine_review_utils import load_clean_data

df = load_clean_data()
assert df.equals(cleaned_reviews[mask][cols]), 'DataFrames are not equal!'
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 98460 entries, 0 to 129970
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   winery       98460 non-null  object 
 1   variety      98460 non-null  object 
 2   year         98460 non-null  int64  
 3   country      98460 non-null  object 
 4   price        91633 non-null  float64
 5   taster_name  98460 non-null  object 
 6   description  98460 non-null  object 
 7   points       98460 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 6.8+ MB


,winery,variety,year,country,price,taster_name,description,points
0,Nicosia,White Blend,2013,Italy,NaN,Kerin O’Keefe,"Aromas include tropical fruit, broom, brimston...",87
1,Quinta dos Avidagos,Portuguese Red,2011,Portugal,15.0,Roger Voss,"This is ripe and fruity, a wine that is smooth...",87
2,Rainstorm,Pinot Gris,2013,US,14.0,Paul Gregutt,"Tart and snappy, the flavors of lime flesh and...",87
3,St. Julian,Riesling,2013,US,13.0,Alexander Peartree,"Pineapple rind, lemon pith and orange blossom ...",87
4,Sweet Cheeks,Pinot Noir,2012,US,65.0,Paul Gregutt,"Much like the regular bottling from 2012, this...",87


In [30]:
# make sure all the values are the same
df.compare(cleaned_reviews[mask][cols])

Empty DataFrame
Columns: []
Index: []

## Next
- [Exploratory Data Analysis](exploratory-data-analysis.ipynb)